In [34]:
import numpy as np
from IPython import display
import torch
from torch.utils import data
from torch import nn
import torch.optim as optim
from tqdm import tqdm
import argparse
from utils import Logger

ImportError: cannot import name 'Logger'

In [9]:
class TimeSeriesChunkDataset(data.Dataset):
    def __init__(self, x, y, context):
        super(TimeSeriesChunkDataset, self).__init__()
        self.x = x
        self.y = y
        self.context = context
        self.points_per_series = self.x.shape[1] - self.context + 1
        
    def __len__(self):
        return self.x.shape[0] * self.points_per_series

    def __getitem__(self, index):
        index_series = index // self.points_per_series
        index_point = index % self.points_per_series
        return_x = self.x[index_series, index_point:index_point+self.context, :]
        return_y = np.argmax(self.y[index_series, :])
        return return_x, return_y

In [11]:
class Discriminator(nn.Module):
    '''
    credit: from https://github.com/martinmamql/complex_da
    '''
    def __init__(self, , d_out):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(

            nn.Linear(feature_dim, feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),
        ) 
        self.fc = nn.Linear(3200, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x: [bs, seq, feature_dim]
        x = self.net(x)
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        out = self.sigmoid(self.fc(x))
        return out

In [12]:
class Generator(nn.Module):
    def __init__(self, **kwargs):
        super(Generator, self).__init__()
        self.transformer_layer_args = {'d_model':2, 'nhead':1, 'dim_feedforward':1024, 'dropout':0.1, 'activation':'gelu'}
        self.transformer_args = {'num_layers':3, 'norm':None}
        self.transformer_layer_args.update(kwargs['transformer_layer'])
        self.transformer_args.update(kwargs['transformer'])
        
        self.transformer_layer = nn.TransformerEncoderLayer(**self.transformer_layer_args)
        self.transformer = nn.TransformerEncoder(self.transformer_layer, **self.transformer_args)
        
    def forward(self, x):
        out = self.transformer(x)
        return out
        

In [13]:
def source_out(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def target_out(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

NameError: name 'data_loader' is not defined

In [ ]:
def train_discriminator(optimizer, trans_source_data, trans_target_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on source Data
    prediction_source = discriminator_GAN(trans_source_data)
    # Calculate error and backpropagate
    error_source = loss(prediction_source, source_out(trans_source_data.size(0)))
    error_source.backward()

    # 1.2 Train on Fake Data
    prediction_target = discriminator_GAN(trans_target_data)
    # Calculate error and backpropagate
    error_target = loss(prediction_target, target_out(trans_target_data.size(0)))
    error_target.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_source + error_target, prediction_source, prediction_target


In [ ]:
def train_generator(optimizer, target_data):
    # 2. Train Generator
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator_GAN(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, source_out(prediction.size(0)))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

In [7]:
#local only

class fake_args():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

        
args = fake_args(data_path='./data_unzip/', 
                 task='3A', 
                 batch_size=30,
                 epochs=2,
                 lr=1e-3,
                 context=10)

args.task = "processed_file_3Av2.pkl" if args.task == "3A" else "processed_file_3E.pkl"
args.data_path = args.data_path + args.task

In [24]:
if __name__ == "__main__":
    cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    data_dict = np.load(args.data_path, allow_pickle=True)
    
    # TODO: to be commented
    data_dict['tr_data'] = data_dict['tr_data'][:10]
    data_dict['tr_data'] = data_dict['tr_data'][:10]
    
    
    data_dict['te_data'] = data_dict['te_data'][:10]
    data_dict['te_data'] = data_dict['te_data'][:10]
    
    # split train data and validation data for source data
    np.random.seed(seed=0)
    indices = np.random.permutation(data_dict['tr_data'].shape[0])
    source_train_x = data_dict['tr_data'][indices[:int(indices.shape[0]*0.9)],:,:].astype("float32")
    source_train_y = data_dict['tr_lbl'][indices[:int(indices.shape[0]*0.9)],:].astype("float32")
    source_vali_x = data_dict['tr_data'][indices[int(indices.shape[0]*0.9):],:,:].astype("float32")
    source_vali_y = data_dict['tr_lbl'][indices[int(indices.shape[0]*0.9):],:].astype("float32")
    
    source_train_dataset = TimeSeriesChunkDataset(source_train_x, source_train_y, args.context)
    source_train_dataloader = data.DataLoader(source_train_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    source_vali_dataset = TimeSeriesChunkDataset(source_vali_x, source_vali_y, args.context)
    source_vali_dataloader = data.DataLoader(source_vali_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    
    # split train data and validation data for target data
    np.random.seed(seed=0)
    indices = np.random.permutation(data_dict['te_data'].shape[0])
    target_train_x = data_dict['te_data'][indices[:int(indices.shape[0]*0.9)],:,:].astype("float32")
    target_train_y = data_dict['te_lbl'][indices[:int(indices.shape[0]*0.9)],:].astype("float32")
    target_vali_x = data_dict['te_data'][indices[int(indices.shape[0]*0.9):],:,:].astype("float32")
    target_vali_y = data_dict['te_lbl'][indices[int(indices.shape[0]*0.9):],:].astype("float32")
    
    target_train_dataset = TimeSeriesChunkDataset(target_train_x, target_train_y, args.context)
    target_train_dataloader = data.DataLoader(target_train_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    target_vali_dataset = TimeSeriesChunkDataset(target_vali_x, target_vali_y, args.context)
    target_vali_dataloader = data.DataLoader(target_vali_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    
    model_args = {
        'classifier': {
            'layers_size': [args.context*2, 100, 100], 
            'dim_out': 50 if args.task == '3A' else 65
        },
        'generator':{
            'transformer_layer': {},
            'transformer': {}
        }, 
        'discriminator': {
            'feature_dim': args.context*2, ## ?
            'd_out': 50 if args.task == '3A' else 65
        }
    }

In [25]:
# Discriminator and Generator
discriminator_GAN = Discriminator(**(model_args["discriminator"]))
generator_GAN = Generator(**(model_args["generator"]))
if torch.cuda.is_available():
    discriminator_GAN.cuda()
    generator_GAN.cuda()

In [27]:

# Optimizers
d_optimizer_GAN = optim.Adam(discriminator_GAN.parameters(), lr=0.0002)
g_optimizer_GAN = optim.Adam(generator_GAN.parameters(), lr=0.0002)

# Loss function
loss = nn.BCELoss()

# Number of steps to apply to the discriminator
d_steps = 1  # In Goodfellow et. al 2014 this variable is assigned to 1
# Number of epochs
num_epochs = 2
num_batches = len(source_train_dataloader)

In [28]:
logger = Logger(model_name='Time-Series-GAN', data_name='Time-Series-Data')



for epoch in range(num_epochs):
    #for n_batch, (source_batch,_) in enumerate(source_train_dataloader):
    for batches, (x_batch, y_batch) in tqdm(enumerate(train_dataloader), total=num_batches):    
        

        # 1. Train Discriminator
        trans_source_data = Variable(x_batch) ### get from the previous transformer
        
        
        
        if torch.cuda.is_available(): trans_source_data = trans_source_data.cuda()
        # Generate fake data
        trans_target_data = generator_GAN(x_batch).detach() ## ? from generator get trans_target_data
        # Train D
        d_error, d_pred_source, d_pred_target = train_discriminator(d_optimizer_GAN,
                                                                trans_source_data, trans_target_data)

        # 2. Train Generator
        # Generate fake data
        # target_data = generator_GAN(??) # get data from generator transformer
        
        # Train G
        g_error = train_generator(g_optimizer_GAN, trans_target_data)
        # Log error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)

        # Display Progress
        if (n_batch) % 100 == 0:
            display.clear_output(True)
            # Display Images
            # test_images = vectors_to_images(generator(test_noise)).data.cpu()
            # logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches);
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_source, d_pred_target
            )
        # Model Checkpoints
        logger.save_models(generator_GAN, discriminator_GAN, epoch)

NameError: name 'Logger' is not defined